## FC Layer Quantitative Analysis

In [2]:
# Import libraries

import os
os.chdir("sandbox-DnD")

import clip
import utils

from sentence_transformers import SentenceTransformer

import pandas as pd

from evaluate import load

In [3]:
# Define variables

clip_name = 'ViT-B/16'
device = 'cuda'
batch_size = 200

In [4]:
# Define embedding models

mpnetmodel = SentenceTransformer('all-mpnet-base-v2')
clip_model, _ = clip.load(clip_name, device=device)
bertscore = load("bertscore")

### Load Labels

In [5]:
# Get ground truths

with open('data/imagenet_labels.txt', 'r') as f:
    classes = f.read().split('\n')

In [6]:
# Get DnD labels

dnd_labels = pd.read_csv('data/DnD_results/rn50_results/resnet50_imagenet_broden_fc.csv')
dnd_preds = list(dnd_labels['label'])

In [7]:
# Get MILAN labels

milan_labels = pd.read_csv('data/MILAN_results/m_base_resnet50_imagenet.csv')
milan_preds = milan_labels[milan_labels['layer']=='fc']
milan_preds = milan_preds.sort_values(by=['unit'])
milan_preds = list(milan_preds['description'])

### Find Similarities

In [9]:
# Calculate similarities between labels and ground truths

clip_cos, mpnet_cos = utils.get_cos_similarity(dnd_preds, classes, clip_model, mpnetmodel, device, batch_size)
bert_score = bertscore.compute(predictions=dnd_preds, references=classes, lang="en")
print("DnD - Clip similarity: {:.4f}, mpnet similarity: {:.4f}, BERTScore: {:4f}".format(clip_cos, mpnet_cos, sum(bert_score["f1"]) / len(bert_score["f1"])))
clip_cos, mpnet_cos = utils.get_cos_similarity(milan_preds, classes, clip_model, mpnetmodel, device, batch_size)
bert_score = bertscore.compute(predictions=milan_preds, references=classes, lang="en")
print("MILAN - Clip similarity: {:.4f}, mpnet similarity: {:.4f}, BERTScore: {:4f}".format(clip_cos, mpnet_cos, sum(bert_score["f1"]) / len(bert_score["f1"])))

DnD - Clip similarity: 0.7598, mpnet similarity: 0.4586, BERTScore: 0.828626
MILAN - Clip similarity: 0.7080, mpnet similarity: 0.2788, BERTScore: 0.820643
